In [27]:
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
#A code to run ordinary least squares with associated statistics
#Jeremy Kedziora
#24 March 2016
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

#import libraries
import numpy as np    #import for arrays
from scipy.optimize import minimize    #import for optimization
import scipy.stats
import statsmodels as sm
from scipy.stats import t


In [28]:
#define functions
def maker(N,n_vars,kind = 'linear',n_cat = 0):
    """A function to generate Monte Carlo linear regression data"""
    x = []    #an empty list to hold the data
    y = np.zeros(N)    #an array to hold the dependent variable
    b = []    #an empty list to hold the true bs
    mu = [0.0]    #an empty list to hold the true cutoffs in the ordered probit
    i = 1
    while i <= n_vars:    #loop over the variables we want to create
        x_i = np.random.normal(loc = 0.0, scale = 1.0, size = N)    #generate the data
        x.append(x_i)    #add it to the list of data
        b_i = np.random.normal(loc = 0.0, scale = 1.0)    #draw a random effect for this variable
        b.append(b_i)    #add it to the list of effects
        y = y + b_i*x_i    #add the variable effect to the dependent variable
        i += 1    #index up i
    
    x.append(np.ones(N))    #and a column of ones for a constant
    b_i = np.random.uniform(0.0,1.0)    #draw a random intercept

    if kind == 'linear':
        y = b_i + y + np.random.normal(loc = 0.0, scale = 1.0, size = N)    #add the normally distributed error term and the intercept
    if kind == 'logit':
        y = (np.random.uniform(0,1,len(y)) < np.exp(b_i + y)/(1 + np.exp(b_i + y)))*1    #draw y values
    if kind == 'ordered':
        y = b_i + y + np.random.normal(loc = 0.0, scale = 1.0, size = N)    #add the normally distributed error term and the intercept
        for i in range(n_cat-2):    #loop over number of categories
            mu.append(mu[i] + np.random.uniform(0.25,1,1)[0])    #append the cutoff for the next category
        y_cat = (y<mu[0])*0    #set the ys less than 0 to category 0
        for i in range(1,len(mu)):    #loop over the remaining categories
            y_cat = y_cat + (mu[i-1]<y)*(y<mu[i])*i    #code all the ys that fall between them
        y_cat = y_cat + (y>mu[len(mu)-1])*(n_cat - 1)    #and code all the ys larger than the largest
        y = y_cat    #and save
    if kind == 'count':
        b_i = np.random.uniform(-1.0,0)    #draw a random intercept
        y = np.random.poisson(np.exp(b_i + y))    #draw the counts
    b.append(b_i)    #append this intercept to the effects
    return [np.array(x).T,np.array(y),np.array(b),np.array(mu)]

In [29]:
def poisson_mle(b,X,y):
    """A function to compute the poisson log-likelihood."""
    xb = X.dot(b)    #compute xb
    return -1*sum(y*xb - np.exp(xb))    #compute the log-likelihood

In [30]:
def ordered_probit_mle(b,X,y):
    """A function to compute the ordered probit log-likelihood."""
    xb = X.dot(b[0:(X.shape[1])])    #compute xb
    mu = [float('-inf'),0]    #initialize the list of mus
    for i in range(len(b[X.shape[1]:])):    #loop over categories
        mu = mu + [mu[i+1] + b[X.shape[1]:][i]]    #and create each mu
    mu = mu + [float('inf')]    #append infinity on the end
    probs = np.zeros(len(y))    #set up an array of 0s
    for i in range(1,len(set(y)) + 1):    #loop over categories
        probs = probs + (scipy.stats.norm.cdf(mu[i] - xb) - scipy.stats.norm.cdf(mu[i - 1] - xb))*(y == list(set(y))[i-1])    #compute probability
    return -1*sum(np.log(probs))

In [37]:
N = 100
n_vars = 3
Data = maker(N,n_vars,'count')    #make poisson data
X = Data[0]    #pull out the features
y = Data[1]    #pull out the labels

b = np.random.uniform(0,1,4)*0.01    #set starting values
model = minimize(poisson_mle, x0 = b, args = (X,y), method = 'BFGS')    #maximize the log-likelihood
Coefficients = model.x    #pull out the coefficients
SE = np.diag(model.hess_inv)**0.5
t_stat = Coefficients/SE    #compute the t statistics for each variable
p_values = 2*t.pdf(abs(t_stat),df = N - 1)    #compute the p-values

print('Beta estimated by MLE:      ',Coefficients)
print('Beta used to generate data: ',Data[2])
print(p_values)

Beta estimated by MLE:       [-0.18315886  0.62673498  0.85678992 -0.76198332]
Beta used to generate data:  [-0.23071022  0.66526728  0.86814841 -0.8919839 ]
[  2.30043220e-01   2.24761510e-06   5.86006111e-07   9.48835362e-05]


In [35]:
N = 1000
n_vars = 3
Data = maker(N,n_vars,'ordered',n_cat=3)    #make poisson data
X = Data[0]    #pull out the features
y = Data[1]    #pull out the labels

b = np.array(list(np.random.uniform(0,1,X.shape[1])*0.01) + list(np.random.uniform(0,1.0,len(set(y))-2)))
model = minimize(ordered_probit_mle, x0 = b, args = (X,y), method = 'BFGS')    #maximize the log-likelihood
Coefficients = model.x    #pull out the coefficients
SE = np.diag(model.hess_inv)**0.5
t_stat = Coefficients/SE    #compute the t statistics for each variable
p_values = 2*t.pdf(abs(t_stat),df = N - 1)    #compute the p-values

print('Beta estimated by MLE:      ',Coefficients)
print('Beta used to generate data: ',list(Data[2]) + list(Data[3][1:]))
print(p_values)

Beta estimated by MLE:       [ 124580.05342331  224967.27454416  639867.30225533  468002.48762422
 -500062.63425663]
Beta used to generate data:  [0.28905599203326476, 0.53508539388182164, 1.55885872411638, 0.82693594880086951, 0.72848728209441937]
[ 0.  0.  0.  0.  0.]
